In [2]:
import numpy as np
import pandas as pd

In [3]:
# setting the environment
# if required, pip install python-dotenv
from dotenv import load_dotenv # Python 3.6+
load_dotenv(verbose = True) # configure .env File or set Environment Variables

True

<p style=" text-align:justify; "><b><code>VisualCrossing</code></b> is a simple to use package to retreive data weather data from <a href = "https://www.visualcrossing.com/weather-data">VisualCrossing</a> (website name same as that of the package). The API is so configured, that just 4-5 lines of code is sufficient to retreive data easily. The website features both historic as well as report on weather forecast. For more information, <a href = "https://www.visualcrossing.com/resources/documentation/">check documentation</a> for the same.</p>

<p style=" text-align:justify; ">This document is designed to provide a <i>quick start guide</i> for anyone who wants to test the code. Currently, the code is in <code>Development Status :: 4 - Beta</code>, thus obviously code and functionalities will be upgraded with new features. <b>Note:</b> minimal documentation is currently provided, and will be updated in time.</p>

Get started by `pip install VisualCrossing`, and then import the package as:

In [4]:
import VisualCrossing as vc

# get installed version
print(vc.__version__)

0.1.0-beta


In [6]:
help(vc.API) # check documentation

Help on class API in module VisualCrossing.api:

class API(VisualCrossing.base)
 |  API(date: str, APIKey: str, location: str, **kwargs)
 |  
 |  A python wrapper to fetch weather data from VisualCrossing
 |  
 |  The function is devised keeping in mind - minimal coding approach, and recurrent job-scheduling
 |  for any person. All the keyword arguments that the API accepts are same as that available
 |  in the Visual Crossing API Documentation (check README for more information). The API is configured
 |  with default values, which is easier to understand and retreive. However, the data requires certain
 |  required arguments defined below. The same can also be obtained from `config.json` file, but is not
 |  recomended.
 |  
 |  :param date: Date for which weather data is required. Pass the date in `YYYY-MM-DD` format,
 |               or pass `FORCAST` to get the forecasted data for a particular date.
 |  
 |  :param APIKey: All VisualCrossing API calls are made with an API key asso

<p style=" text-align:justify; ">The API can be configured with a <code>configuration</code> which is located at <code>HOME/.visualCrossing/config.json</code>, and an example file is provided in the directory. The file is set, such that same setting can be reused, or if required you can keep pre-defined templates to be used regularly.</p>

All the attributes required, can be defined like:

```json
...
    "attributes": {
        "unitGroup": "metric",
        "contentType": "dataframe",
        "aggregateHours": 24
    },
...
```

The API requires three minimal options:
1. Date for which the data is required, in `YYYY-MM-DD` format,
2. API Key which can be obtained from [website](https://www.visualcrossing.com/weather-api), and
3. Location for which data is required. The location can be of the following types:
   * Name of a single place, like `"india"`,
   * Coordinates of a place (if data for the coordinate is not available, then nearest location data is returned), this should be in `"latitude,longitude"` format,
   * To retreive location for multiple location, pass the data like:
   ```python
    ["india", "tokyo"] # either passing names,
    ["20.593683,78.962883", "35.689487,139.691711"] # passing list/tuple of coordinates,
    ("india", "35.689487,139.691711") # or even a combination of name or coordinate
   ```
   * _you can also use US ZIP Codes_.

Location is also available by `(lat, lon)` pairs, for which you can also use [this website](https://www.latlong.net/) to find location easily.

In [5]:
locations = ("india", "tokyo")
api = vc.API("2021-09-25", APIKey = os.getenv("__api_key__"), location = locations, contentType = "dataframe") # default parameters are set from `config.json`

In [6]:
data = api.get(ssl_verify = True)
data.head()

../VisualCrossing/api.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  total = np.array([self._get_df_converted_data(loc, ssl_verify) for loc in self._location])


,Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,...,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions
0,india,09/25/2021,25.5,32.5,28.7,25.7,84.75,40.9,9.4,NaN,...,3.2,69.5,1006.3,"Mist, Thunderstorm Without Precipitation, Smok...",28.6310,77.2172,India,India,NaN,"Rain, Partially cloudy"
0,tokyo,09/25/2021,20.4,24.1,22.2,17.2,73.29,NaN,18.1,NaN,...,18.8,25.4,1024.0,"Rain, Light Rain, Sky Coverage Increasing, Smo...",35.6841,139.8090,"Tokyo, Japan","Tokyo, Japan",NaN,Partially cloudy
